## Setup

In [5]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import pandas as pd
#import pyarrow.feather as feather
from sklearn.model_selection import train_test_split 

from tensorflow.keras.callbacks import ReduceLROnPlateau
from sklearn.preprocessing import PowerTransformer
#from tensorflow.keras.layers.experimental import preprocessing


/home/uwe/anaconda3/envs/tf-gpu-cuda8/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/uwe/anaconda3/envs/tf-gpu-cuda8/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/uwe/anaconda3/envs/tf-gpu-cuda8/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/uwe/anaconda3/envs/tf-gpu-cuda8

# W & B test

In [6]:
# import wandb
# from wandb.keras import WandbCallback
# !wandb login


## Prepare the data

In [7]:
# fluxData_df = feather.read_feather('data/fluxData.feather')
# # zernikeData_df = feather.read_feather('data/zernikeData.feather')
# fluxData_df


## import data as h5

In [8]:
fluxData_df =pd.read_hdf("data/fluxData_df.h5", key="fluxData_df")

zernikeData_df =pd.read_hdf("data/zernikeData_df.h5", key="zernikeData_df")

In [9]:
# Prepare the training dataset
x_train = fluxData_df
y_train = zernikeData_df
# x_train, x_test, y_train, y_test = train_test_split(
#     X, y, test_size=0.1, random_state=42)


## Box-Cox transform
turned out to be of no effect

In [10]:
pt = PowerTransformer()

#x_trainTransform = pt.fit_transform(x_train)

## Normalize data
as shown in https://keras.io/guides/preprocessing_layers/#normalizing-numerical-features 

In [11]:

x_train_norm = (x_train - x_train.mean(axis=0)) / x_train.std(axis=0)
x_train_norm

,FluxCoeef1,FluxCoeef2,FluxCoeef3,FluxCoeef4,FluxCoeef5,FluxCoeef6,FluxCoeef7,FluxCoeef8,FluxCoeef9,FluxCoeef10,FluxCoeef11,FluxCoeef12,FluxCoeef13,FluxCoeef14,FluxCoeef15,FluxCoeef16,FluxCoeef17,FluxCoeef18,FluxCoeef19
0,0.900396,-0.312539,-0.772603,-0.136057,-0.515690,1.002374,-1.519562,-1.267677,2.596329,-1.010410,-0.143842,-2.373536,1.105245,2.871782,-0.608565,-0.844337,0.683551,1.348749,-1.338257
1,-0.006083,0.690461,0.706083,0.001087,-0.135921,-0.396939,-0.426242,0.680082,-1.163970,0.370308,-0.359454,-0.103394,-0.670624,0.551241,-0.832305,-0.111359,-0.335083,0.090348,-0.070232
2,0.181662,-1.537365,0.046234,-0.595047,0.581989,-0.771069,0.281238,0.494130,-1.038512,-0.823826,2.898185,1.414300,-0.651543,-1.063024,0.513900,1.057111,-0.846276,-0.285620,0.325633
3,1.104926,-0.493233,1.156159,1.508609,-0.510300,-0.986929,-1.012457,0.279925,0.245774,0.918173,-0.830261,-0.717984,-0.352607,0.253269,0.497768,0.852767,0.561766,-0.343766,-0.816310
4,-2.766798,0.836719,-1.246774,-1.907403,-2.494540,2.514527,1.213441,-0.793285,-1.033874,-0.097034,0.795638,1.403599,0.435049,-0.978962,2.012168,-0.004434,0.344632,0.770427,1.462520
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58735,1.111759,-1.962699,1.064218,0.667639,-1.961451,-1.014588,0.223547,-1.405704,-0.003764,-0.830787,-0.682253,-0.615273,0.168018,-0.899590,0.452977,1.137328,0.070017,3.806415,0.336357
58736,-0.892319,-0.834533,-1.203572,-1.178698,0.789761,0.682359,2.491787,-0.012967,-1.165161,-0.431385,0.405629,-0.740750,-0.911497,0.110287,-0.775899,2.467752,0.814036,-0.972636,1.145398
58737,0.783504,-1.503598,-0.629578,1.116959,1.017882,0.327938,1.402795,-0.098698,-0.405518,0.629009,-1.205222,-0.989608,-1.052450,-1.082164,-0.551245,2.690250,-0.919279,-1.418809,-0.327146
58738,-1.123925,2.829232,0.281349,0.459110,-0.539199,0.814040,-0.162167,0.123279,-0.639389,1.866671,-1.661150,-1.217272,-0.752796,0.835984,-0.637278,-0.090449,2.143275,1.077346,-0.202596


# Build Model

In [ ]:
from tensorflow import keras
AoModel = keras.Sequential([
     layers.InputLayer(19, name="digits"),
     layers.Dense(2000, activation="relu", name = "Layer1"),
     #keras.layers.Dropout(0.5),
     #keras.layers.BatchNormalization(),
     layers.Dense(1050, activation="relu", name="Layer2"),
     #keras.layers.Dropout(0.5),
     #keras.layers.BatchNormalization(),
     layers.Dense(100, activation="relu", name="Layer3"),
     #keras.layers.BatchNormalization(),
     layers.Dense(9, activation="linear", name="predictions"),

])
keras.utils.plot_model(AoModel, "AoModel.png", show_shapes = True)

## Build batchnorm model 

In [ ]:
from tensorflow import keras
AoBatchNormModel = keras.Sequential([
     layers.BatchNormalization(19, name="digits"),
     layers.Dense(2000, activation="relu", name = "Layer1"),
     #keras.layers.Dropout(0.5),
     #keras.layers.BatchNormalization(),
     layers.Dense(1050, activation="relu", name="Layer2"),
     #keras.layers.Dropout(0.5),
     #keras.layers.BatchNormalization(),
     layers.Dense(100, activation="relu", name="Layer3"),
     #keras.layers.BatchNormalization(),
     layers.Dense(9, activation="linear", name="predictions"),

])
#keras.utils.plot_model(AoAoBatchNormModelModel, "AoModel.png", show_shapes = True)
#AoBatchNormModel.summary()


In [ ]:
import tensorflow_addons as tfa
opt = tf.ad

## Build model with funcitonal API

In [ ]:
NoLayers =4
inputs = keras.Input(shape=(19,))
Layer1 = layers.Dense(2000, activation="relu", name = "Layer1")
x = Layer1(inputs)
x = layers.Dense(1050, activation="relu", name = "Layer2")(x)
x = layers.Dense(1000, activation="relu", name = "Layer3")(x)
if(NoLayers >3):
    x = layers.Dense(100, activation="relu", name = "Layer4")(x)
outputs = layers.Dense(9, name = "Output")(x)
AoFunctionalModel = keras.Model(inputs=inputs, outputs=outputs, name="AOModel")

AoFunctionalModel.summary()

In [ ]:
keras.utils.plot_model(AoFunctionalModel, "AoModel.png", show_shapes = True)

## Run Ao funcitonal model

In [ ]:
batch_size = 128
epochs = 200
#tf.random.set_seed(2021)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=5, min_lr=0.000001, verbose=1, cooldown= 5)
opt = keras.optimizers.Adam(learning_rate=0.001)
AoFunctionalModel.compile(loss= keras.losses.MeanSquaredError(),  optimizer=opt, metrics= [tf.keras.metrics.RootMeanSquaredError()])
AoFunctionalModel.fit(x_train_norm, y_train, batch_size=batch_size, epochs=epochs,validation_split = 0.2, verbose = 2, callbacks=[reduce_lr])

#AoModel.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1, callbacks=[WandbCallback()])

## 9 different last hidden layers

In [ ]:
from keras.utils.generic_utils import get_custom_objects
from keras.layers import Activation
from keras import backend as K
def custom_activation(x):
    return (K.sigmoid(x) - 0.5) *2
get_custom_objects().update({'linear': Activation(custom_activation)})


inputs = keras.Input(shape=(19,))
Layer1 = layers.Dense(2000, activation="linear", name = "Layer1")
x = Layer1(inputs)
x = layers.Dense(1050, activation="relu", name = "Layer2")(x)

# create 9 outputs
x1 = layers.Dense(100, activation="relu", name = "Layer3_1")(x)
out1 = layers.Dense(1, activation=custom_activation, name = "out1")(x1)

x2 = layers.Dense(100, activation="relu", name = "Layer3_2")(x)
out2 = layers.Dense(1, activation=custom_activation, name = "out2")(x2)

x3 = layers.Dense(100, activation="relu", name = "Layer3_3")(x)
out3 = layers.Dense(1, activation=custom_activation, name = "out3")(x3)

x4 = layers.Dense(100, activation="relu", name = "Layer3_4")(x)
out4 = layers.Dense(1, activation=custom_activation, name = "out4")(x4)

x5 = layers.Dense(100, activation="relu", name = "Layer3_5")(x)
out5 = layers.Dense(1, activation=custom_activation, name = "out5")(x5)

x6 = layers.Dense(100, activation="relu", name = "Layer3_6")(x)
out6 = layers.Dense(1, activation=custom_activation, name = "out6")(x6)

x7 = layers.Dense(100, activation="relu", name = "Layer3_7")(x)
out7 = layers.Dense(1, activation=custom_activation, name = "out7")(x7)

x8 = layers.Dense(100, activation="relu", name = "Layer3_8")(x)
out8 = layers.Dense(1, activation=custom_activation, name = "out8")(x8)

x9 = layers.Dense(100, activation="relu", name = "Layer3_9")(x)
out9 = layers.Dense(1, activation=custom_activation, name = "out9")(x9)


outputs = tf.keras.layers.Concatenate(axis=1)([out1,out2,out3,out4,out5,out6,out7,out8,out9])


#outputs = layers.Dense(9, name = "Output")(x)
Ao9Exits = keras.Model(inputs=inputs, outputs=outputs, name="AOModel")

Ao9Exits.summary()

In [ ]:
keras.utils.plot_model(AoFunctionalModel, "AoModel.png", show_shapes = True)

## train model

In [ ]:
batch_size = 128
epochs = 200
tf.random.set_seed(2021)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=5, min_lr=0.000001, verbose=1, cooldown= 5)
opt = keras.optimizers.Adam(learning_rate=0.001)
Ao9Exits.compile(loss= keras.losses.MeanSquaredError(),  optimizer=opt, metrics= [tf.keras.metrics.RootMeanSquaredError()])
Ao9Exits.fit(x_train_norm, y_train, batch_size=batch_size, epochs=epochs,validation_split = 0.2, verbose = 2, callbacks=[reduce_lr])

#AoModel.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1, callbacks=[WandbCallback()])

## Build Batchnorm model

In [ ]:
NoLayers =4
inputs = keras.Input(shape=(19,))
BatchNorm1 = layers.BatchNormalization()
Layer1 = layers.Dense(2000, activation="relu", name = "Layer1")
x = BatchNorm1(inputs)
x = Layer1(x)
x = layers.Dense(1050, activation="relu", name = "Layer2")(x)
x = layers.BatchNormalization()(x)
x = layers.Dense(1000, activation="relu", name = "Layer3")(x)
x = layers.BatchNormalization()(x)

if(NoLayers >3):
    x = layers.Dense(100, activation="relu", name = "Layer4")(x)
    x = layers.BatchNormalization()(x)

outputs = layers.Dense(9, name = "Output")(x)
AoBatchNormModel = keras.Model(inputs=inputs, outputs=outputs, name="AOModel")

AoBatchNormModel.summary()

## Train batchNorm model

In [ ]:
batch_size = 128
epochs = 200
#tf.random.set_seed(2021)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=5, min_lr=0.000001, verbose=1, cooldown= 5)
opt = keras.optimizers.Adam(learning_rate=0.001)
AoBatchNormModel.compile(loss= keras.losses.MeanSquaredError(),  optimizer=opt, metrics= [tf.keras.metrics.RootMeanSquaredError()])
AoBatchNormModel.fit(x_train_norm, y_train, batch_size=batch_size, epochs=epochs,validation_split = 0.2, verbose = 2, callbacks=[reduce_lr])

#AoModel.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1, callbacks=[WandbCallback()])

## Build COnv1D model

In [12]:
conv1D_1 =64
conv1D_1_width = 4
conv1D_2 = 32
conv1D_2_width = 2



input_shape = (4, 19, 1)

AoConv1DModel = keras.Sequential([
     
     
     #keras.layers.Conv1D(32, 3, activation='relu',input_shape=(None,19), name="conv1d"),
     keras.layers.Conv1D( conv1D_1, conv1D_1_width, activation='relu',input_shape=input_shape[1:]),
     keras.layers.MaxPool1D(pool_size=2),
     #keras.layers.Dropout(rate = 0.2),
     keras.layers.Conv1D( conv1D_2, conv1D_2_width, activation='relu',input_shape=input_shape[1:]),
     keras.layers.MaxPool1D(pool_size=2),


     #keras.layers.Dropout(rate = 0.2),
     keras.layers.Flatten(),
     keras.layers.Dense(2000, activation="relu"),
     #keras.layers.Dropout(rate = 0.2),
     keras.layers.Dense(1050, activation="relu"),
     #keras.layers.Dropout(rate = 0.1),

     keras.layers.Dense(100, activation="relu"),
     #keras.layers.Dropout(rate = 0.2),

     keras.layers.Dense(9,  name="predictions"),

])
AoConv1DModel.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 16, 64)            320       
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 8, 64)             0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 7, 32)             4128      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 3, 32)             0         
_________________________________________________________________
flatten (Flatten)            (None, 96)                0         
_________________________________________________________________
dense (Dense)                (None, 2000)              194000    
____

## Reshape training data

In [13]:
input_train = x_train_norm.to_numpy()[:,:,np.newaxis]


Train simple model

## Train Conv1D model

In [14]:
batch_size = 128
epochs = 200
#tf.random.set_seed(2021)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=5, min_lr=0.000001, verbose=1, cooldown= 5)
opt = keras.optimizers.Adam(learning_rate=0.001)
AoConv1DModel.compile(loss= keras.losses.MeanSquaredError(),  optimizer=opt, metrics= [tf.keras.metrics.RootMeanSquaredError()])
AoConv1DModel.fit(input_train, y_train, batch_size=batch_size, epochs=epochs,validation_split = 0.2, verbose = 2, callbacks=[reduce_lr])

#AoModel.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1, callbacks=[WandbCallback()])

Train on 46992 samples, validate on 11748 samples
Epoch 1/200
46992/46992 - 3s - loss: 0.0068 - root_mean_squared_error: 0.0822 - val_loss: 0.0039 - val_root_mean_squared_error: 0.0621
Epoch 2/200
46992/46992 - 3s - loss: 0.0026 - root_mean_squared_error: 0.0509 - val_loss: 0.0023 - val_root_mean_squared_error: 0.0482
Epoch 3/200
46992/46992 - 3s - loss: 0.0019 - root_mean_squared_error: 0.0436 - val_loss: 0.0019 - val_root_mean_squared_error: 0.0432
Epoch 4/200
46992/46992 - 3s - loss: 0.0015 - root_mean_squared_error: 0.0390 - val_loss: 0.0019 - val_root_mean_squared_error: 0.0441
Epoch 5/200
46992/46992 - 3s - loss: 0.0014 - root_mean_squared_error: 0.0368 - val_loss: 0.0016 - val_root_mean_squared_error: 0.0400
Epoch 6/200
46992/46992 - 3s - loss: 0.0012 - root_mean_squared_error: 0.0344 - val_loss: 0.0017 - val_root_mean_squared_error: 0.0417
Epoch 7/200
46992/46992 - 3s - loss: 0.0011 - root_mean_squared_error: 0.0330 - val_loss: 0.0016 - val_root_mean_squared_error: 0.0398
Epoch

KeyboardInterrupt: 